In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import os
from tqdm.notebook import tqdm
import re

# --- 1. 設定項目 ---
ANNOTATION_LIST_CSV = '../data/ground_truth/annotation_target_list.csv' # or your final 200-item list
MASTER_LIST_CSV = '../data/processed/citing_papers_with_paths.csv'
OUTPUT_DIR = '../data/processed'
OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'samples_with_text.csv')

# XMLの名前空間の定義
NAMESPACES = {
    'ce': 'http://www.elsevier.com/xml/common/dtd',
    'ja': 'http://www.elsevier.com/xml/ja/dtd',
    'dc': 'http://purl.org/dc/elements/1.1/',
    'core': 'http://www.elsevier.com/xml/svapi/article/dtd',
    'xocs': 'http://www.elsevier.com/xml/xocs/dtd'
}

# --- 2. 堅牢な抽出関数群（最終版） ---

def extract_abstract_robustly(root):
    """
    XMLのroot要素から、判明した全てのパターンを試してアブストラクトを抽出する。
    """
    try:
        # パターン1: 本文内の詳細なアブストラクト (`<ce:abstract class="author">`)
        paras = root.findall('.//ja:article/ja:head/ce:abstract[@class="author"]//ce:simple-para', NAMESPACES)
        if paras:
            text = ' '.join(p.text.strip() for p in paras if p.text)
            if text: return ' '.join(text.split())

        # パターン2: 一般的なアブストラクト (`<ce:abstract>`)
        paras = root.findall('.//ce:abstract/ce:abstract-sec//ce:simple-para', NAMESPACES)
        if paras:
            text = ' '.join(p.text.strip() for p in paras if p.text)
            if text: return ' '.join(text.split())

        # パターン3: メタデータ内のアブストラクト (`<dc:description>`)
        description = root.find('.//core:coredata/dc:description', NAMESPACES)
        if description is not None and description.text:
            text = description.text.strip()
            if text: return ' '.join(text.split())
            
    except Exception:
        return None
    return None

def extract_full_text_robustly(root):
    """
    XMLのroot要素から、図表や不要セクション、数式などを除いたクリーンな全文テキストを抽出する。
    """
    try:
        full_text_parts = []
        excluded_keywords = ['acknowledgement', 'references', 'bibliography', 'author contribution', 'competing interest', 'funding']

        # パターンA: 構造化された本文 (`<ja:body><ce:sections>`)
        body_sections = root.find('.//ja:body/ce:sections', NAMESPACES)
        if body_sections is not None:
            for section in body_sections.findall('.//ce:section', NAMESPACES):
                title_tag = section.find('./ce:section-title', NAMESPACES)
                sec_title = title_tag.text.strip().lower() if title_tag is not None and title_tag.text else ''
                
                if not any(keyword in sec_title for keyword in excluded_keywords):
                    for para in section.findall('./ce:para', NAMESPACES):
                        para_text_parts = [para.text.strip()] if para.text else []
                        for child in para:
                            if child.tag not in [f'{{{NAMESPACES["ce"]}}}formula', f'{{{NAMESPACES["ce"]}}}display']:
                                if child.tail: para_text_parts.append(child.tail.strip())
                        
                        clean_text = ' '.join(' '.join(part for part in para_text_parts if part).split())
                        if clean_text: full_text_parts.append(clean_text)
            if full_text_parts:
                return " ".join(full_text_parts)

        # パターンB: 非構造化テキスト (`<rawtext>`)
        raw_text_element = root.find('.//xocs:doc/xocs:rawtext', NAMESPACES)
        if raw_text_element is not None and raw_text_element.text:
            raw_text = raw_text_element.text
            return ' '.join(raw_text.split())

    except Exception:
        return None
    return None

# --- 3. メインの処理ループ ---
try:
    df_targets = pd.read_csv(ANNOTATION_LIST_CSV)
    df_targets.drop_duplicates(subset=['citing_paper_doi'], inplace=True)
    df_master = pd.read_csv(MASTER_LIST_CSV)
    df_master_subset = df_master[['citing_paper_doi', 'fulltext_xml_path']].drop_duplicates(subset=['citing_paper_doi'])
    df_to_process = pd.merge(df_targets, df_master_subset, on='citing_paper_doi', how='left')
    
    extracted_data_list = []
    print(f"処理対象の {len(df_to_process)} 件の論文からテキストを抽出します...")
    
    for index, row in tqdm(df_to_process.iterrows(), total=len(df_to_process), desc="テキスト抽出中"):
        xml_path = row.get('fulltext_xml_path')
        abstract = None
        full_text = None
        
        if pd.notna(xml_path) and os.path.exists(xml_path):
            try:
                tree = ET.parse(xml_path)
                root = tree.getroot()
                abstract = extract_abstract_robustly(root)
                full_text = extract_full_text_robustly(root)
            except ET.ParseError:
                print(f"警告: DOI {row['citing_paper_doi']} のXMLをパースできませんでした。")

        extracted_data_list.append({
            'citing_paper_eid': row['citing_paper_eid'],
            'citing_paper_doi': row['citing_paper_doi'],
            'citing_paper_title': row['citing_paper_title'],
            'cited_data_paper_title': row['cited_data_paper_title'],
            'abstract': abstract,
            'full_text': full_text
        })

    # --- 4. 最終的なCSVファイルの保存 ---
    df_final = pd.DataFrame(extracted_data_list)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    df_final.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    
    print(f"\n処理完了。")
    print(f"抽出したテキストを '{OUTPUT_FILE}' に保存しました。")
    print("\n--- 監査を実行します ---")
    print(df_final.info())

except Exception as e:
    print(f"\nメイン処理中にエラーが発生しました: {e}")

処理対象の 200 件の論文からテキストを抽出します...


テキスト抽出中:   0%|          | 0/200 [00:00<?, ?it/s]


処理完了。
抽出したテキストを '../data/processed\samples_with_text.csv' に保存しました。

--- 監査を実行します ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   citing_paper_eid        200 non-null    object
 1   citing_paper_doi        200 non-null    object
 2   citing_paper_title      200 non-null    object
 3   cited_data_paper_title  200 non-null    object
 4   abstract                198 non-null    object
 5   full_text               200 non-null    object
dtypes: object(6)
memory usage: 9.5+ KB
None
